In [ ]:
# Make an object which can be initiallized (to load all the nessecary models etc so it doesnt
# have to be done each time). Make individual class detector methods.
# Make a predict class pipeline that goes through the flowchart,
# initiating each class detector when needed.

In [1]:
#Import data
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
data = pd.read_pickle('PPEAP_test.pkl') #Load test data
data.loc[data['label'] == 6 ,'label'] = 2 # hotfix
#ohe = OneHotEncoder().fit(data.pred.to_numpy().reshape(-1,1)) # Deze pickelen?
import pickle
# with open("answer_ohe", "wb") as f: 
#     pickle.dump(ohe, f)




In [9]:
import math
from pytorch_pretrained_bert import OpenAIGPTTokenizer, OpenAIGPTModel, OpenAIGPTLMHeadModel
from PIL import Image
import tensorflow as tf
from pathlib import Path
import torch

class Explanation_error_detector:
    
    def __init__(self,perplexity_threshold=137, model_pickles = ['Information_error_model.tf','AE_Mismatch_model.tf','answer_ohe']):
        ### Text Encoder:
        # Load Language Model
        # Load pre-trained model (weights)
        self.model = OpenAIGPTLMHeadModel.from_pretrained('openai-gpt')
        self.model.eval()
        # Load pre-trained model tokenizer (vocabulary)
        self.tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')
        
        ### Img Encoder:
        # Not used for the moment to save time
#         self.image_features_extract_model = tf.keras.applications.Xception(
#             include_top=True,
#             weights="imagenet",
#             input_tensor=None,
#             input_shape=None,
#             pooling=None,
#             classes=1000,
#         )
        
        ### Detection models:
        self.Inf_Err_model = tf.keras.models.load_model(model_pickles[0])
        self.AE_model = tf.keras.models.load_model(model_pickles[1])
        
        ### One Hot Encoder for answers:
        self.one_hot = pickle.load(open(model_pickles[2], 'rb'))
        
        ### For clarity error:
        self.perplexity_threshold = perplexity_threshold
        
    def perplexity_score(self,sentence):
        if type(sentence) != str:
            sentence = sentence.to_string()
        tokenize_input = self.tokenizer.tokenize(sentence)
        tensor_input = torch.tensor([self.tokenizer.convert_tokens_to_ids(tokenize_input)])
        loss=self.model(tensor_input, lm_labels=tensor_input)
        return math.exp(loss)
    
    # Doesn't work yet: assume features have been pre-generated for now.
#     def encode_image(self,img_path):
#             img = tf.io.read_file(img_path)
#             img = tf.image.decode_jpeg(img, channels=3)
#             img_pre = tf.image.resize(img, (299, 299))
#             img = tf.keras.applications.xception.preprocess_input(img_pre)
#             img = tf.reshape(img,(1,299,299,3),output)
#             print(img)
#             img = self.image_features_extract_model(img).numpy()
#             return img
    
    def detect_clarity_error(self,explanation):
       # Kan je vgm ook een probability van maken op een bepaalde manier maar eh
        if self.perplexity_score(explanation)>self.perplexity_threshold:
            return True
        else:
            return False
    def detect_information_error(self, explanation,image, npy=True):
        
        #print(explanation.shape, image.shape)
        # If npy=True --> Explanation and image should be numpy arrays containing the encodings
        pred = self.Inf_Err_model.predict([np.vstack(image),np.vstack(explanation)])
        return pred
        # Transform image into encodings (skipped) -> predict using IE model -> Return probab (only return label in full model)
        
    def detect_AE_error(self, explanation, qstr, ans, npy=True): # Note: ans is pred from dataset
        # If npy=True --> explanation and question should be numpy arrays containing the encodings
        pred = self.AE_model.predict([np.vstack(qstr), np.vstack(explanation)])
        pred_ans = self.one_hot.inverse_transform(pred)
        
        if type(ans) != str:
            ans = ans.to_string()

            if (pred_ans[0][0] == ans):
                return 0
            else:
                return 1
        else:
            if (pred_ans[0][0] == ans):
                return 0
            else:
                return 1
        
    def detect_error(self,explanation, exp_enc, image, qstr, pred, ans, label = 0, npy= True):
        
        # If label is given, ground_truth model is activated: the data is inserted in the first model
        # that can correctly determine the label. This simulates a situation in which all models
        # previous to the relevant one for the data point are perfect (thus the datapoint has not been
        # falsely classified before it reached the model that should classify it correctly)
        
        #print('true label:',label)
        if type(pred) == str:
            answer_match = pred == ans
        else:
            answer_match = (pred == ans).bool()
        if (self.detect_clarity_error(explanation) and label <=1) or exp_enc.shape[2] > 22:
            #print('label <=1:',label)
            return 1
        elif self.detect_information_error(exp_enc, image)>0.5 and label <=2:
            #print('label <=2:',label)
            return 2
        elif self.detect_AE_error(exp_enc, qstr, pred) and not label in (3,5):
            #print('label not 3 or 5:',label)
            return 4
        elif answer_match and not label == 3: # answer correct
            #print('label not 3:',label)
            return 5
        else:
            return 3 # reasoning error

    def detect_individual(self,explanation, exp_enc, image, qstr, pred, ans, label = 0, npy= True):
        # Make a 4*4 matrix, 
        mat = np.zeros((4,4))
        if exp_enc.shape[2] > 22:
            mat[1] = np.array([0,1,0,0]) # prevent error from enc's that are too big
            return mat
        
        clar_error = 1 if self.detect_clarity_error(explanation) else 0
        inf_error = 1 if (self.detect_information_error(exp_enc, image)>0.5)[0][0] else 0
        ae_error = 1 if bool(self.detect_AE_error(exp_enc, qstr, pred)) else 0 
        
        if sum((clar_error,inf_error,ae_error)) == 0:
            no_error = 1
        else:
            no_error = 0
        

        if label in (5,3,6):
            mat[0] = np.array([no_error, clar_error, inf_error, ae_error])
            return mat
        elif label == 4:
            mat[3] = np.array([no_error, clar_error, inf_error, ae_error])
            return mat
        else:
            mat[label] = np.array([no_error, clar_error, inf_error, ae_error])
            return mat


    def pad_text_npy(self,text):
        s = text.shape
        if s[1] >22:
            np.resize(text,(s[0],22,s[2]))
            out = text
        else:
            out = np.pad(text, ((0,0),(0,22-s[1]),(0,0)),'constant', constant_values = 0)

        return out
        
    def set_perplexity_threshold(self, t):
        self.perplexity_threshold = t

In [10]:
eer = Explanation_error_detector(perplexity_threshold = 150)
#print(eer.encode_image(str(Path('D:\\Users\\Jokmenen\\Documents\\MEGA\\Uni\Master\\Thesis\\SethParkResults\\coco\\COCO_val2014_000000532732.jpg'))))

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [5]:
data.exp_enc = data.exp_enc.apply(eer.pad_text_npy)
data.qstr_enc = data.qstr_enc.apply(eer.pad_text_npy)

In [29]:
ploenkie = data.sample(5)

#print("INF ONLY:", eer.detect_information_error(ploenkie.exp_enc, ploenkie.img_enc))
# ploenkie_pred = ploenkie.apply(lambda x:
#                                eer.detect_error(x.exp, np.array([x.exp_enc]), np.array([x.img_enc]),
#                                                 np.array([x.qstr_enc]), x.pred, x.ans, x.label), axis=1)


#eer.detect_error(ploenkie.exp, ploenkie.exp_enc, ploenkie.img_enc, ploenkie.qstr_enc, ploenkie.pred, ploenkie.ans)
# pred_label = 5 if (eer.one_hot.inverse_transform(ploenkie_pred)[0][0] == ploenkie.ans).bool() else 4
# pred_label, ploenkie.label

#print('pred:',ploenkie_pred, 'true:',ploenkie.label)

#eer.detect_clarity_error(ploenkie.exp)

bliep = ploenkie.apply(lambda x:
                               eer.detect_individual(x.exp, np.array([x.exp_enc]), np.array([x.img_enc]),
                                                np.array([x.qstr_enc]), x.pred, x.ans, x.label), axis=1).values
print(np.sum(bliep), bliep)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [3. 1. 2. 1.]
 [0. 0. 0. 0.]] [array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 0.]])
 array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 0.]])
 array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 0.]])
 array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 1., 1., 1.],
       [0., 0., 0., 0.]])
 array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.]])]


## Full Model

In [11]:
%%time

#DIT NOG EEN KEER DOEN MET GOEIE TEST/TRAIN VERDELING
data['pred_label_full'] = data.apply(lambda x:
                               eer.detect_error(x.exp, np.array([x.exp_enc]), np.array([x.img_enc]),
                                                np.array([x.qstr_enc]), x.pred, x.ans), axis=1)

Wall time: 37.3 s


In [12]:
yhat = data.pred_label_full
y = data.label

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(classification_report(y,yhat))
print(confusion_matrix(y,yhat))
print(accuracy_score(y,yhat))

              precision    recall  f1-score   support

           1       0.18      0.67      0.28        48
           2       0.61      0.34      0.44       206
           3       0.12      0.22      0.15         9
           4       0.09      0.22      0.13        23
           5       0.69      0.44      0.54       206

    accuracy                           0.40       492
   macro avg       0.34      0.38      0.31       492
weighted avg       0.57      0.40      0.44       492

[[32 11  0  2  3]
 [59 70 14 26 37]
 [ 3  4  2  0  0]
 [16  2  0  5  0]
 [67 27  1 21 90]]
0.40447154471544716


## Real Model Ground Truth

In [32]:
data

#DIT NOG EEN KEER DOEN MET GOEIE TEST/TRAIN VERDELING
data['pred_label_full_gt'] = data.apply(lambda x:
                               eer.detect_error(x.exp, np.array([x.exp_enc]), np.array([x.img_enc]),
                                                np.array([x.qstr_enc]), x.pred, x.ans, x.label), axis=1)

In [33]:
yhat = data.pred_label_full_gt
y = data.label

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(classification_report(y,yhat))
print(confusion_matrix(y,yhat))
print(accuracy_score(y,yhat))

#Recall is hier belangrijk: 
#    van de keren dat je categorie x moest vinden, hoe vaak vond je hem ook daadwerkelijk

#Precision:
# van de keren dat je categorie x gevonden hebt, hoe vaak was het ook echt categorie x
# Niet relevant want voor de eerdere categorieën is dat altijd hoger dan voor de latere


              precision    recall  f1-score   support

           1       1.00      0.69      0.81        48
           2       0.90      0.50      0.64       206
           3       0.30      1.00      0.46         9
           4       0.25      0.61      0.35        23
           5       0.79      0.99      0.88       206

    accuracy                           0.74       492
   macro avg       0.65      0.76      0.63       492
weighted avg       0.82      0.74      0.74       492

[[ 33  11   0   1   3]
 [  0 103  17  42  44]
 [  0   0   9   0   0]
 [  0   0   1  14   8]
 [  0   0   3   0 203]]
0.7357723577235772


## Segmented Models

In [14]:
seg_matrices = data.apply(lambda x:
                               eer.detect_individual(x.exp, np.array([x.exp_enc]), np.array([x.img_enc]),
                                                np.array([x.qstr_enc]), x.pred, x.ans, x.label), axis=1)



In [15]:
seg_matrix_summed = np.sum(seg_matrices)

print(seg_matrix_summed)
#print(seg_matrices)

[[ 90.  78.  46.  61.]
 [  3.  33.  31.  28.]
 [ 47.  67. 103. 101.]
 [  0.  17.  12.  14.]]


In [119]:
np.zeros((4,3))

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [198]:
for enc,exp in zip(data.exp_enc, data.exp):
    if not enc.shape[1] == len(exp.split()):
        print(exp, enc.shape[1], len(exp.split()) )

it is cut in a slices and full of toppings 11 10
there are trains and a common hydrant in the background 11 10
the sign says dixie hwy and thats in the united states 13 11
he is on a surfboard 6 5
there is a stove and refrigarator and a sink in it 13 11
it has wings and is perched on a tree brance 11 10
he is on a snowboard on the snow 9 8
the sky is blue and there isnt a cloud in the sky 13 12
he is on a snowboard on the snow 9 8
he is on a snowboard on the snow 9 8
they are wearing uniforms and tuxedos 7 6
the colors and the racquets are dull 8 7
his hair is brushing his shirt colar 8 7
it is a macbook 5 4
it has a retractable handle 6 5
he is wearing a helmet and is snowboarding 9 8
he is on a surfboard 6 5
they are on a snowboard in the snow 9 8
the giraffe is in a pen 7 6
they are on skateboards 5 4
he is on a snowboard on the snow 9 8
they are on a tennis court with rackets and tennis balls 12 11
it is a round round piece of marbled meat in a dish 13 12
he is on a snowboard on th

In [199]:
m = 0 #Find size of biggest explanation to potentially use in model as cutoff point. (all longer explanations are probably clarity errors.) 
for x in data.exp[data.label == 5]:
    l = len(x.split())
    if l>m:
        m = l
m

17

In [6]:

# for col in ('exp_enc','qstr_enc')
# data.exp_enc = data.exp_enc.apply(pad_exp)
data.exp_enc.iloc[3].shape


(1, 22, 768)

In [203]:
t = eer.one_hot.transform(np.array('yes').reshape(-1,1)).todense()

eer.one_hot.inverse_transform(t)

array([['yes']], dtype=object)

## Find best threshold on train set:

In [40]:
from sklearn.model_selection import train_test_split, KFold
from tqdm import tqdm

train = pd.read_pickle('PPEAP_train.pkl') #use train to determine threshold

train.exp_enc = train.exp_enc.apply(eer.pad_text_npy)
train.qstr_enc = train.qstr_enc.apply(eer.pad_text_npy)

kf = KFold(n_splits=5)
results = {}

fold = 1

for train_index, _ in kf.split(train):
    res_list = []
    print(train_index)
    cv_train = train.iloc[train_index]
    for t in tqdm(range(137, 200, 5)):
        eer.set_perplexity_threshold(t)
        cv_train['pred_label_full_gt'] = cv_train.apply(lambda x:
                           eer.detect_error(x.exp, np.array([x.exp_enc]), np.array([x.img_enc]),
                            np.array([x.qstr_enc]), x.pred, x.ans, x.label), axis=1)

        yhat = cv_train.pred_label_full_gt
        y = cv_train.label
        print(yhat, y)
        print(accuracy_score(y,yhat))
        res_list += accuracy_score(y,yhat)
    results[fold] = res_list
    fold += 1






  0%|                                                                                                                                                       | 0/13 [00:00<?, ?it/s]

[ 296  297  298 ... 1473 1474 1475]


c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  8%|██████████▉                                                                                                                                   | 1/13 [01:51<22:13, 111.15s/it]

9729001      3
249883001    5
577887003    3
120007009    5
484978002    2
497299002    5
455448001    1
574790005    2
416309001    2
5352001      5
305464001    5
101656002    5
50025002     2
245450002    2
566576002    2
198416000    2
318219015    2
460461007    5
90839010     5
363319001    5
506614001    5
134255001    5
84241003     5
523883004    2
527215000    5
214197002    5
214832000    5
554523043    3
439015003    5
206878001    1
            ..
82835003     2
29431001     5
91643001     5
201072002    5
225405002    5
332027005    3
15542001     2
168898001    3
379882001    5
247057002    5
416758002    5
92700000     5
307658003    2
54092002     4
12120001     5
557812009    3
322574002    2
547583002    2
343466025    5
351868003    2
269678027    5
291404001    5
118366006    2
177861002    5
457817004    5
243626001    4
466835001    5
304099003    2
30290002     2
448535001    5
Name: pred_label_full_gt, Length: 1180, dtype: int64 9729001      2
249883001    5
57

c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 15%|█████████████████████▊                                                                                                                        | 2/13 [03:40<20:18, 110.73s/it]

...

180447001    2
296969006    1
200717011    5
157807002    2
304744000    5
131539008    5
415218000    5
168593017    2
538888001    2
146813008    2
171717004    5
68291003     2
4021001      5
262476000    1
120592002    2
283516001    4
38576002     5
184778002    2
147787002    2
76416004     5
547783005    5
549112001    1
203372006    2
260172000    2
130118002    1
438253001    1
304389003    5
572069002    2
245660002    2
6213003      5
            ..
273493011    2
567877001    5
468197002    5
93511007     5
44800003     5
38034000     2
460243001    5
535506003    4
239985003    2
130262000    5
494998001    3
24530000     5
61943005     4
479829001    5
104801009    3
189267005    5
356920001    2
420059001    5
536894000    2
441840013    2
49115002     4
522430002    5
128014004    4
551648000    2
409241003    5
442726000    2
526044021    5
345294001    5
487990002    5
83142002     2
Name: pred_label_full_gt, Length: 1181, dtype: int64 180447001    2
296969006    1
20

c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 92%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏          | 12/13 [20:04<01:40, 100.25s/it]

180447001    2
296969006    1
200717011    5
157807002    2
304744000    5
131539008    5
415218000    5
168593017    2
538888001    2
146813008    2
171717004    5
68291003     2
4021001      5
262476000    1
120592002    2
283516001    4
38576002     5
184778002    2
147787002    2
76416004     5
547783005    5
549112001    1
203372006    2
260172000    2
130118002    1
438253001    1
304389003    5
572069002    2
245660002    2
6213003      5
            ..
273493011    2
567877001    5
468197002    5
93511007     5
44800003     5
38034000     2
460243001    5
535506003    4
239985003    2
130262000    5
494998001    3
24530000     5
61943005     4
479829001    5
104801009    3
189267005    5
356920001    2
420059001    5
536894000    2
441840013    2
49115002     4
522430002    5
128014004    4
551648000    2
409241003    5
442726000    2
526044021    5
345294001    5
487990002    5
83142002     2
Name: pred_label_full_gt, Length: 1181, dtype: int64 180447001    2
296969006    1
20

c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [21:55<00:00, 101.23s/it]

180447001    2
296969006    1
200717011    5
157807002    2
304744000    5
131539008    5
415218000    5
168593017    2
538888001    2
146813008    2
171717004    5
68291003     2
4021001      5
262476000    1
120592002    2
283516001    4
38576002     5
184778002    2
147787002    2
76416004     5
547783005    5
549112001    1
203372006    2
260172000    2
130118002    1
438253001    1
304389003    5
572069002    2
245660002    2
6213003      5
            ..
273493011    2
567877001    5
468197002    5
93511007     5
44800003     5
38034000     2
460243001    5
535506003    4
239985003    2
130262000    5
494998001    3
24530000     5
61943005     4
479829001    5
104801009    3
189267005    5
356920001    2
420059001    5
536894000    2
441840013    2
49115002     4
522430002    5
128014004    4
551648000    2
409241003    5
442726000    2
526044021    5
345294001    5
487990002    5
83142002     2
Name: pred_label_full_gt, Length: 1181, dtype: int64 180447001    2
296969006    1
20

In [41]:
results


{1: array([], dtype=float64),
 2: array([], dtype=float64),
 3: array([], dtype=float64),
 4: array([], dtype=float64),
 5: array([], dtype=float64)}